### Getter und Setter
Es können aber auch eigene Getter und Setter für jedes Feld definiert werden, falls der Variablentyp dies zulässt und diese erst im Rumpf der Klasse definiert werden. Beide müssen direkt nach dem Feld definiert werden. Auf den aktuellen Wert des Feldes kann mit `field` zugegriffen werden. Der Setter bekommt den neuen Wert als Parameter übergeben. Der Datentyp des Parameters muss nicht angegeben werden, da dieser durch das Feld gegeben ist, jedoch muss ihm ein Name zugewiesen werden, der im Rumpf den neuen Wert repräsentiert. Der Wert des Feldes wird mit einer Zuweisung an `field` geändert. Es sind immer die Datentypen der Felder zu beachten.

In [12]:
public class Person (name: String, alter: Int){
    val name: String = name
        get() = "Name: $field" //Wird auf das Feld name zugegeriffen, wird der String "Name: name" zurückgegeben
        //Kein Setter möglich, da val
    var alter = if (alter > 0) alter else -1
        get() = field //Standardgetter
        set(value){ //Eigener Setter. Der neue Wert wird in value gespeichert
            field = if (value > field) value else field //Das Feld wird auf den neuen Wert gesetzt, falls dieser größer als der Aktuelle ist.
        }
}
val p1 = Person ("Max Mustermann", -10)
println("Person p1: ${p1.name}, Alter ${p1.alter}")
p1.alter = 21
println("Person p1: ${p1.name}, Alter ${p1.alter}")
p1.alter = -10
println("Person p1: ${p1.name}, Alter ${p1.alter}")

Person p1: Name: Max Mustermann, Alter -1
Person p1: Name: Max Mustermann, Alter 21
Person p1: Name: Max Mustermann, Alter 21


#### Computed Properties
Durch die Definition eigener Getter können Felder auch *kleine Methoden* darstellen ohne einen eigenen Wert zu speichern. So können beispielsweise berechnete Wahrheitswerte zurückgegeben werden. Dadurch können rudimentäre Methoden ersetzt werden. Als Beispiel wird die bereits kennengelernte Klasse `Person` mit einem berechneten Feld `istVolljaerig` ergänzt.

In [13]:
public class Person (name: String, alter: Int){
    val name: String = name
        get() = "Name: $field" //Wird auf das Feld name zugegeriffen, wird der String "Name: name" zurückgegeben
        //Kein Setter möglich, da val
    var alter = if (alter > 0) alter else -1
        get() = field //Standardgetter
        set(value){ //Eigener Setter. Der neue Wert wird in value gespeichert
            field = if (value > field) value else field //Das Feld wird auf den neuen Wert gesetzt, falls dieser größer als der Aktuelle ist.
        }
    val istVolljaerig //Datentyp Boolean aus dem Getter
        get() = alter > 17
}
val p1 = Person ("Max Mustermann", 21)
println("Person p1: ${p1.name}, Alter ${p1.alter}, Volljährig ${p1.istVolljaerig}")

Person p1: Name: Max Mustermann, Alter 21, Volljährig true


#### Lazy Properties
Eine Variable oder ein Feld des Typs `val` kann mit dem Schlüsselwort `by lazy` *träge* gemacht werden. Das heißt, dass der Wert erst bei der ersten Verwendung berechnet wird. Dazu ist ein Lambda-Ausdruck nötig, dessen Ergebnis der Wert des Feldes oder der Variable ist. Wird die Variable danach noch einmal verwendet, wird der Inhalt nicht noch einmal berechnet, sondern auf den gespeicherten Wert zurückgegriffen.

In [14]:
public class Person (name: String, alter: Int){
    val name = name
    var alter = alter
    val istVolljaehrig: Boolean by lazy {
        println("Wird berechnet...")
        alter > 18
    }
}
val p1 = Person("Erika Musterfrau", 32)
println(p1.istVolljaehrig)
println(p1.istVolljaehrig)

Wird berechnet...
true
true


#### `lateinit`
Ein weiteres, ähnliches Konstrukt ist `lateinit`. Dies findet Anwendung, wenn ein Feld erst nach der Objekterzeugung einen Wert zugewiesen bekommt. Dieses Feld kann im Konstruktor ignoriert werden.  
Vorraussetzungen:
* Das Feld ist vom Typ `var`
* Der Datentyp des Feldes ist *non-nullable*
* Der Datentyp des Feldes ist nicht primitiv

In [15]:
public class Person (name: String, alter: Int){
    val name = name
    var alter = alter
    lateinit var istVolljaehrig: String //istVolljaehrig wird nicht bei Objekterzeugung berechnet
    
    fun berechneIstVolljaehrig() { //Diese Methode berechnet den Wert des Feldes istVolljaehrig
        istVolljaehrig = if (alter > 17) "Volljährig" else "Nicht Volljährig"
    }
}
val p1 = Person ("Max Mustermann", 30)
//println(p1.istVolljaehrig) -> Fehler: UninitializedPropertyAccessException: lateinit property istVolljaehrig has not been initialized
p1.berechneIstVolljaehrig() //istVolljaehrig wird berechnet
println(p1.istVolljaehrig)

Volljährig


In einem `lateinit` Feld ist zusätzlich gespeichert, ob es bereits für das Objekt initialisiert wurde. Diese Information kann mit `this::name.isInitialized` abgerufen werden.

In [16]:
public class Person (name: String, alter: Int){
    val name = name
    var alter = alter
    lateinit var istVolljaehrig: String
    
    fun berechneIstVolljaehrig(){
        istVolljaehrig = if (alter > 17) "Volljährig" else "Nicht Volljährig"
    }
    fun istInitialisiert() = this::istVolljaehrig.isInitialized
}

val p1 = Person ("Max Mustermann", 30)
println("Versuch 1: ${p1.istInitialisiert()}")
p1.berechneIstVolljaehrig() //istVolljaehrig wird berechnet
println("Versuch 2: ${p1.istInitialisiert()}")

Versuch 1: false
Versuch 2: true


### Sichtbarkeiten
Bis jetzt wurden alle Variablen, Felder und Klassen öffentlich implementiert. Jedoch gibt es in Kotlin, wie auch in Java, Sichtbarkeiten, mit denen der Zugriff eingeschränkt werden kann.
* `private`: Das Element kann nur in der Klasse verwendet werden. 
* `protected`: Erweitertung von `private`. Zusätzlich können auf die Elemente auch in Unterklassen zugegriffen werden.
* `internal`: Erweiterung von `protected`. Das Element kann im ganzen Modul verwendet werden. Module können Sie sich als größere Pakete vorgestellen.
* `public`: Auf das Element kann überall zugegriffen werden. Standardsichtbarkeit.

Bei der automatischen Generierung der Getter und Setter werden auch ihre Sichtbarkeiten der des Felder angeglichen. Ist das Feld `private`, kann auf deren generierten Getter und Setter nicht außerhalb der Klasse zugegriffen werden.

In [140]:
class PrivatePerson(private val name: String)
val p1 = PrivatePerson("test")
p1.name

Line_159.jupyter-kts (3:4 - 8) Cannot access 'name': it is private in 'Person'

Werden die Methoden händisch implementiert, können diesen teilweise abweichende Sichtbarkeiten zugewiesen werden. Der Getter muss den identischen Sichtbarkeitstyp aufweisen. Im Gegensatz dazu kann dem Setter die gleich oder eine *engere* Sichtbarkeit vorangestellt werden.

In [151]:
class LesbarePerson(name: String) {
    var name = name //public
    get() = field //public
    private set(neuerWert){ //private
        field = neuerWert
    }
}
val p2 = LesbarePerson("test")
println("Name von p2: ${p2.name}")
p2.name = "Name"

Line_170.jupyter-kts (10:1 - 8) Cannot assign to 'name': the setter is private in 'LesbarePerson'

<div class="alert alert-block alert-info">
Im Folgenden werden alle Klassen, Felder und Methoden in der Regel öffentlich implementiert. Auch wird in den kurzen Aufgaben dieses Abschnitts keine Verwendung einer anderen Sichtbarkeit als <code>public</code> verlangt. Sie können aber natürlich gerne optional in den Aufgaben die Zugriffe mit passenden Berechtigungen für die Getter und Setter beschränken.
</div>

### Methoden
Der Aufbau von Objektmethoden kann mit dem von Funktionen verglichen werden, werden jedoch im Rumpf einer Klasse definiert. Es ist möglich die Kurzschreibweise anzuwenden und eine Sichtbarkeit zuzuweisen. Auf Objektmethoden kann, wie in Java, mit der Punktnotation zugegriffen werden. Soll eine bereits implementierte Methode, zum Beispiel `toString()`, überschrieben werden, ist das Voranstellen des Schlüsselworts `override` nötig.

In [17]:
class Person(val name: String, var alter: Int){
    fun hatGeburtstag() = alter++ //Objektmethode
    override fun toString() = "$name ist $alter Jahre alt." //Objektmethode die die Methode toString() überschreibt
}

val p1 = Person("Max Mustermann", 30)
println("Vor Geburtstag: $p1")
p1.hatGeburtstag()
println("Nach Geburtstag: $p1")

Vor Geburtstag: Max Mustermann ist 30 Jahre alt.
Nach Geburtstag: Max Mustermann ist 31 Jahre alt.
